In [1]:
import os
from dotenv import load_dotenv, find_dotenv
dotenv_path = os.path.abspath('../../../.env')
_ = load_dotenv(dotenv_path)
OPENAI_API_KEY  = os.environ['OPENAI_API_KEY']
PG_DB_PW = os.environ['PG_DB_PW']
host= os.environ['HOST']
port= os.environ['PORT']
user= os.environ['USER']
password= os.environ['PG_DB_PW']
dbname= os.environ['DBNAME']

In [2]:
from langchain.vectorstores.pgvector import PGVector
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import ChatOpenAI, OpenAI


In [3]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector


/var/folders/s4/l2klswfj1wvcw_vbmvvbc70h0000gn/T/ipykernel_37462/1185487198.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import psycopg2
conn = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=PG_DB_PW,
    database=dbname
)

OperationalError: connection to server at "172.17.0.2", port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?


In [33]:
# Create a cursor object to execute SQL commands
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

In [30]:
# Register the vector type with psycopg2
register_vector(conn)


In [31]:
# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE embeddings (
            id bigserial primary key, 
            title text,
            url text,
            content text,
            tokens integer,
            embedding vector(1536)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()



In [18]:
# Setup database
db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{user}:{PG_DB_PW}@{host}:{port}/{dbname}",
)

In [21]:
# setup llm
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name='gpt-3.5-turbo')

# Create query instruction
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

{question}
"""

# Setup the database chain
db_chain = SQLDatabaseChain.from_llm(OpenAI(), db, verbose=True)


In [22]:
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [23]:
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

{question}
"""

In [10]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)


/Users/dangause/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [11]:
def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)
                break


In [13]:
get_prompt()

Type 'exit' to quit




> Entering new SQLDatabaseChain chain...

Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

how many tasks do i have?

SQLQuery:SELECT COUNT(*) FROM tasks;
SQLResult: [(15,)]
Answer:Final answer here: You have 15 tasks.
> Finished chain.
Final answer here: You have 15 tasks.


> Entering new SQLDatabaseChain chain...

Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"



SQLQuery:SELECT task, due_date 
FROM tasks 
WHERE completed = 0 
ORDER BY priority DESC 
LIMIT 5;
SQLResult: [('Clean house', datetime.d

In [39]:
CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}?sslmode=require"


In [42]:
# Setup database
db = SQLDatabase.from_uri(
    CONNECTION_STRING,
)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: server does not support SSL, but SSL was required

(Background on this error at: https://sqlalche.me/e/20/e3q8)